<a href="https://colab.research.google.com/github/nebyu08/data_sc/blob/main/deployment_of_public_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi -q
!pip install uvicorn -q
!pip install pickles -q
!pip install pydantic -q
!pip install scikit-learn -q
!pip install requests -q
!pip install pypi-json -q
!pip install nest-asyncio -q
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=3c22a3928faf953ecc73226bf20b5abd27b5eff2e1ba700ddcbd5dae9794add1
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [3]:
app=FastAPI()

In [4]:
origins=["*"]

In [5]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
   )

In [6]:
class model_input(BaseModel):
  Pregnancies:int
  Glucose: int
  BloodPressure: int
  SkinThickness: int
  Insulin : int
  BMI : float
  DiabetesPedigreeFunction : float
  Age : int


In [7]:
 #loading the saved model
model=pickle.load(open("/content/model.pkl","rb"))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
@app.post('/diabetic_prediction')
def diabetic_prediction(input_parameters:model_input):
  input_data=input_parameters.json() #turn the data into json format
  input_dictionary=json.loads(input_data) # turn the data from json into python dictionary format

  preg=input_dictionary['Pregnancies']
  glu=input_dictionary['Glucose']
  bp=input_dictionary['BloodPressure']
  skin=input_dictionary['SkinThickness']
  ins=input_dictionary['Insulin']
  bmi=input_dictionary['BMI']
  dpf=input_dictionary['DiabetesPedigreeFunction']
  age=input_dictionary['Age']

  input_list=[preg,glu,bp,skin,ins,bmi,dpf,age]

  prediction=model.predict([input_list])
  if (prediction[0] ==0 ) :
    return "the person isnot non diabetic"
  else:
    return "the person is diabetic"


In [ ]:
# lets form the public URL
ngrok_tunnel=ngrok.connect(8000)
print("the public url is:",ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=8000)

INFO:     Started server process [885]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


the public url is: https://6d5d-35-197-77-172.ngrok.io
INFO:     213.55.73.204:0 - "POST /diabetic_prediction HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
